# How to create a custom checkpointer using Redis

When creating LangGraph agents, you can also set them up so that they persist their state. This allows you to do things like interact with an agent multiple times and have it remember previous interactions. Make sure that you have Redis running on port `6379` for going through this tutorial

This example shows how to use `Redis` as the backend for persisting checkpoint state.

NOTE: this is just an example implementation. You can implement your own checkpointer using a different database or modify this one as long as it conforms to the `BaseCheckpointSaver` interface.

## Install the necessary libraries for Redis on Python

In [1]:
%%capture --no-stderr
%pip install -U redis langgraph langchain_openai


[notice] A new release of pip is available: 24.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## Checkpointer implementation

In [2]:
"""Implementation of a langgraph checkpoint saver using Redis."""
import logging
from contextlib import asynccontextmanager, contextmanager
from typing import Any, AsyncGenerator, Generator, List, Optional, Tuple, Union
import redis
from redis.asyncio import ConnectionPool as AsyncConnectionPool
from redis.asyncio import Redis as AsyncRedis
from datetime import datetime, timezone

from langchain_core.runnables import RunnableConfig
from langgraph.checkpoint.base import BaseCheckpointSaver, Checkpoint, CheckpointMetadata, CheckpointTuple
from langgraph.checkpoint.serde.jsonplus import JsonPlusSerializer

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Serializer class
class JsonAndBinarySerializer(JsonPlusSerializer):
    """Serializer to handle both JSON and binary data."""
    def _default(self, obj: Any) -> Any:
        if isinstance(obj, (bytes, bytearray)):
            return self._encode_constructor_args(
                obj.__class__, method="fromhex", args=[obj.hex()]
            )
        return super()._default(obj)

    def dumps(self, obj: Any) -> str:
        """Serialize object to a JSON string."""
        try:
            if isinstance(obj, (bytes, bytearray)):
                return obj.hex()
            return super().dumps(obj)
        except Exception as e:
            logger.error(f"Serialization error: {e}")
            raise

    def loads(self, s: str, is_binary: bool = False) -> Any:
        """Deserialize object from a JSON string."""
        try:
            if is_binary:
                return bytes.fromhex(s)
            return super().loads(s)
        except Exception as e:
            logger.error(f"Deserialization error: {e}")
            raise

# Connection initialization
def initialize_sync_pool(host: str = "localhost", port: int = 6379, db: int = 0, **kwargs) -> redis.ConnectionPool:
    """Initialize a synchronous Redis connection pool."""
    try:
        pool = redis.ConnectionPool(host=host, port=port, db=db, **kwargs)
        logger.info(f"Synchronous Redis pool initialized with host={host}, port={port}, db={db}")
        return pool
    except Exception as e:
        logger.error(f"Error initializing sync pool: {e}")
        raise

def initialize_async_pool(url: str = "redis://localhost", **kwargs) -> AsyncConnectionPool:
    """Initialize an asynchronous Redis connection pool."""
    try:
        pool = AsyncConnectionPool.from_url(url, **kwargs)
        logger.info(f"Asynchronous Redis pool initialized with url={url}")
        return pool
    except Exception as e:
        logger.error(f"Error initializing async pool: {e}")
        raise

# Connection handling context managers
@contextmanager
def _get_sync_connection(connection: Union[redis.Redis, redis.ConnectionPool, None]) -> Generator[redis.Redis, None, None]:
    """Context manager for managing synchronous Redis connections."""
    conn = None
    try:
        if isinstance(connection, redis.Redis):
            yield connection
        elif isinstance(connection, redis.ConnectionPool):
            conn = redis.Redis(connection_pool=connection)
            yield conn
        else:
            raise ValueError("Invalid sync connection object.")
    except redis.ConnectionError as e:
        logger.error(f"Sync connection error: {e}")
        raise
    finally:
        if conn:
            conn.close()

@asynccontextmanager
async def _get_async_connection(connection: Union[AsyncRedis, AsyncConnectionPool, None]) -> AsyncGenerator[AsyncRedis, None]:
    """Context manager for managing asynchronous Redis connections."""
    conn = None
    try:
        if isinstance(connection, AsyncRedis):
            yield connection
        elif isinstance(connection, AsyncConnectionPool):
            conn = AsyncRedis(connection_pool=connection)
            yield conn
        else:
            raise ValueError("Invalid async connection object.")
    except redis.ConnectionError as e:
        logger.error(f"Async connection error: {e}")
        raise
    finally:
        if conn:
            await conn.aclose()

# Redis operations
def _save_to_redis(conn, key: str, data: dict):
    """Save data to Redis."""
    try:
        conn.hset(key, mapping=data)
        logger.info(f"Data stored successfully under key: {key}")
    except Exception as e:
        logger.error(f"Failed to save data to Redis: {e}")
        raise

async def _save_to_redis_async(conn, key: str, data: dict):
    """Asynchronously save data to Redis."""
    try:
        await conn.hset(key, mapping=data)
        logger.info(f"Data stored successfully under key: {key}")
    except Exception as e:
        logger.error(f"Failed to save data to Redis: {e}")
        raise

def _get_redis_data(conn, key: str) -> Optional[dict]:
    """Retrieve data from Redis."""
    try:
        data = conn.hgetall(key)
        if data:
            logger.info(f"Data retrieved successfully for key: {key}")
            return data
        else:
            logger.info(f"No valid data found for key: {key}")
            return None
    except Exception as e:
        logger.error(f"Failed to retrieve data from Redis: {e}")
        raise

async def _get_redis_data_async(conn, key: str) -> Optional[dict]:
    """Asynchronously retrieve data from Redis."""
    try:
        data = await conn.hgetall(key)
        if data:
            logger.info(f"Data retrieved successfully for key: {key}")
            return data
        else:
            logger.info(f"No valid data found for key: {key}")
            return None
    except Exception as e:
        logger.error(f"Failed to retrieve data from Redis: {e}")
        raise

# Main class
class RedisSaver(BaseCheckpointSaver):
    """Redis-based checkpoint saver implementation."""
    sync_connection: Optional[Union[redis.Redis, redis.ConnectionPool]] = None
    async_connection: Optional[Union[AsyncRedis, AsyncConnectionPool]] = None

    def __init__(
        self,
        sync_connection: Optional[Union[redis.Redis, redis.ConnectionPool]] = None,
        async_connection: Optional[Union[AsyncRedis, AsyncConnectionPool]] = None,
    ):
        super().__init__(serde=JsonAndBinarySerializer())
        self.sync_connection = sync_connection
        self.async_connection = async_connection

    def put(
        self,
        config: RunnableConfig,
        checkpoint: Checkpoint,
        metadata: CheckpointMetadata,
    ) -> RunnableConfig:
        """Synchronously store a checkpoint in Redis."""
        thread_id = config["configurable"]["thread_id"]
        parent_ts = config["configurable"].get("thread_ts")
        key = f"checkpoint:{thread_id}:{checkpoint['ts']}"
        data = {
            "checkpoint": self.serde.dumps(checkpoint),
            "metadata": self.serde.dumps(metadata),
            "parent_ts": parent_ts if parent_ts else "",
        }
        with _get_sync_connection(self.sync_connection) as conn:
            _save_to_redis(conn, key, data)
        return {"configurable": {"thread_id": thread_id, "thread_ts": checkpoint["ts"]}}

    async def aput(
        self,
        config: RunnableConfig,
        checkpoint: Checkpoint,
        metadata: CheckpointMetadata,
    ) -> RunnableConfig:
        """Asynchronously store a checkpoint in Redis."""
        thread_id = config["configurable"]["thread_id"]
        parent_ts = config["configurable"].get("thread_ts")
        key = f"checkpoint:{thread_id}:{checkpoint['ts']}"
        data = {
            "checkpoint": self.serde.dumps(checkpoint),
            "metadata": self.serde.dumps(metadata),
            "parent_ts": parent_ts if parent_ts else "",
        }
        async with _get_async_connection(self.async_connection) as conn:
            await _save_to_redis_async(conn, key, data)
        return {"configurable": {"thread_id": thread_id, "thread_ts": checkpoint["ts"]}}

    def put_writes(
        self,
        config: RunnableConfig,
        writes: List[Tuple[str, Any]],
        task_id: str,
    ) -> RunnableConfig:
        """Synchronously save a list of writes to Redis storage."""
        thread_id = config["configurable"]["thread_id"]
        thread_ts = config["configurable"].get("thread_ts") or self._default_ts()
        key = f"writes:{thread_id}:{thread_ts}"
        serialized_writes = [(channel, self.serde.dumps(value)) for channel, value in writes]
        with _get_sync_connection(self.sync_connection) as conn:
            for channel, value in serialized_writes:
                conn.rpush(key, f"{task_id}:{channel}:{value}")
        logger.info(f"Writes stored successfully for thread_id: {thread_id}, ts: {thread_ts}")
        return config

    async def aput_writes(
        self,
        config: RunnableConfig,
        writes: List[Tuple[str, Any]],
        task_id: str,
    ) -> RunnableConfig:
        """Asynchronously save a list of writes to Redis storage."""
        thread_id = config["configurable"]["thread_id"]
        thread_ts = config["configurable"].get("thread_ts") or self._default_ts()
        key = f"writes:{thread_id}:{thread_ts}"
        serialized_writes = [(channel, self.serde.dumps(value)) for channel, value in writes]
        async with _get_async_connection(self.async_connection) as conn:
            for channel, value in serialized_writes:
                await conn.rpush(key, f"{task_id}:{channel}:{value}")
        logger.info(f"Writes stored successfully for thread_id: {thread_id}, ts: {thread_ts}")
        return config

    def get_tuple(self, config: RunnableConfig) -> Optional[CheckpointTuple]:
        """Retrieve a checkpoint tuple from Redis."""
        thread_id = config["configurable"]["thread_id"]
        thread_ts = config["configurable"].get("thread_ts")
        with _get_sync_connection(self.sync_connection) as conn:
            key = self._get_checkpoint_key(conn, thread_id, thread_ts)
            if not key:
                return None
            checkpoint_data = _get_redis_data(conn, key)
            return self._parse_checkpoint_data(checkpoint_data, config, thread_id)

    async def aget_tuple(self, config: RunnableConfig) -> Optional[CheckpointTuple]:
        """Asynchronously retrieve a checkpoint tuple from Redis."""
        thread_id = config["configurable"]["thread_id"]
        thread_ts = config["configurable"].get("thread_ts")
        async with _get_async_connection(self.async_connection) as conn:
            key = await self._aget_checkpoint_key(conn, thread_id, thread_ts)
            if not key:
                return None
            checkpoint_data = await _get_redis_data_async(conn, key)
            return self._parse_checkpoint_data(checkpoint_data, config, thread_id)

    def list(
        self,
        config: Optional[RunnableConfig],
        *,
        filter: Optional[dict[str, Any]] = None,
        before: Optional[RunnableConfig] = None,
        limit: Optional[int] = None,
    ) -> Generator[CheckpointTuple, None, None]:
        """List checkpoints from Redis."""
        thread_id = config["configurable"]["thread_id"] if config else "*"
        pattern = f"checkpoint:{thread_id}:*"
        with _get_sync_connection(self.sync_connection) as conn:
            keys = self._filter_keys(conn, pattern, before, limit)
            for key in keys:
                data = _get_redis_data(conn, key)
                if data and b"checkpoint" in data and b"metadata" in data:
                    yield self._parse_checkpoint_data(data, config, thread_id)

    async def alist(
        self,
        config: Optional[RunnableConfig],
        *,
        filter: Optional[dict[str, Any]] = None,
        before: Optional[RunnableConfig] = None,
        limit: Optional[int] = None,
    ) -> AsyncGenerator[CheckpointTuple, None]:
        """Asynchronously list checkpoints from Redis."""
        thread_id = config["configurable"]["thread_id"] if config else "*"
        pattern = f"checkpoint:{thread_id}:*"
        async with _get_async_connection(self.async_connection) as conn:
            keys = await self._afilter_keys(conn, pattern, before, limit)
            for key in keys:
                data = await _get_redis_data_async(conn, key)
                if data and b"checkpoint" in data and b"metadata" in data:
                    yield self._parse_checkpoint_data(data, config, thread_id)

    # Utility methods
    def _default_ts(self) -> str:
        """Generate a default timezone-aware timestamp."""
        return datetime.now(timezone.utc).isoformat()

    def _get_checkpoint_key(self, conn, thread_id: str, thread_ts: Optional[str]) -> Optional[str]:
        """Determine the Redis key for a checkpoint."""
        if thread_ts:
            return f"checkpoint:{thread_id}:{thread_ts}"
        all_keys = conn.keys(f"checkpoint:{thread_id}:*")
        if not all_keys:
            logger.info(f"No checkpoints found for thread_id: {thread_id}")
            return None
        latest_key = max(all_keys, key=lambda k: ":".join(k.decode().split(":")[2:]))
        return latest_key.decode()

    async def _aget_checkpoint_key(self, conn, thread_id: str, thread_ts: Optional[str]) -> Optional[str]:
        """Asynchronously determine the Redis key for a checkpoint."""
        if thread_ts:
            return f"checkpoint:{thread_id}:{thread_ts}"
        all_keys = await conn.keys(f"checkpoint:{thread_id}:*")
        if not all_keys:
            logger.info(f"No checkpoints found for thread_id: {thread_id}")
            return None
        latest_key = max(all_keys, key=lambda k: ":".join(k.decode().split(":")[2:]))
        return latest_key.decode()

    def _filter_keys(self, conn, pattern: str, before: Optional[RunnableConfig], limit: Optional[int]) -> list:
        """Filter and sort Redis keys based on optional criteria."""
        keys = conn.keys(pattern)
        if before:
            keys = [
                k
                for k in keys
                if ":".join(k.decode().split(":")[2:])
                < before["configurable"]["thread_ts"]
            ]
        keys = sorted(keys, key=lambda k: ":".join(k.decode().split(":")[2:]), reverse=True)
        if limit:
            keys = keys[:limit]
        return keys

    async def _afilter_keys(self, conn, pattern: str, before: Optional[RunnableConfig], limit: Optional[int]) -> list:
        """Asynchronously filter and sort Redis keys based on optional criteria."""
        keys = await conn.keys(pattern)
        if before:
            keys = [
                k
                for k in keys
                if ":".join(k.decode().split(":")[2:])
                < before["configurable"]["thread_ts"]
            ]
        keys = sorted(keys, key=lambda k: ":".join(k.decode().split(":")[2:]), reverse=True)
        if limit:
            keys = keys[:limit]
        return keys

    def _parse_checkpoint_data(self, data: dict, config: RunnableConfig, thread_id: str) -> Optional[CheckpointTuple]:
        """Parse checkpoint data retrieved from Redis."""
        if not data:
            logger.info(f"No valid checkpoint data found.")
            return None
        checkpoint = self.serde.loads(data[b"checkpoint"].decode())
        metadata = self.serde.loads(data[b"metadata"].decode())
        parent_ts = data.get(b"parent_ts", b"").decode()
        parent_config = (
            {"configurable": {"thread_id": thread_id, "thread_ts": parent_ts}}
            if parent_ts
            else None
        )
        logger.info(f"Checkpoint parsed successfully for thread_id: {thread_id}")
        return CheckpointTuple(
            config=config,
            checkpoint=checkpoint,
            metadata=metadata,
            parent_config=parent_config,
        )


## Checkpointer implementation

## Setup environment

In [3]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

## Setup model and tools for the graph

In [4]:
from typing import Literal
from langchain_core.runnables import ConfigurableField
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent


@tool
def get_weather(city: Literal["nyc", "sf"]):
    """Use this to get weather information."""
    if city == "nyc":
        return "It might be cloudy in nyc"
    elif city == "sf":
        return "It's always sunny in sf"
    else:
        raise AssertionError("Unknown city")


tools = [get_weather]
model = ChatOpenAI(model_name="gpt-4o", temperature=0)

## Use sync connection

### With a connection pool

In [5]:
sync_pool = initialize_sync_pool(host="localhost", port=6379, db=0)

INFO:__main__:Synchronous Redis pool initialized with host=localhost, port=6379, db=0


In [6]:
checkpointer = RedisSaver(sync_connection=sync_pool)

In [7]:
graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
config = {"configurable": {"thread_id": "1"}}
res = graph.invoke({"messages": [("human", "what's the weather in sf")]}, config)

INFO:__main__:No checkpoints found for thread_id: 1
INFO:__main__:Data stored successfully under key: checkpoint:1:2024-08-05T11:15:21.936463+00:00
INFO:__main__:Writes stored successfully for thread_id: 1, ts: 2024-08-05T11:15:21.947055+00:00
INFO:__main__:Data stored successfully under key: checkpoint:1:2024-08-05T11:15:21.946487+00:00
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Data stored successfully under key: checkpoint:1:2024-08-05T11:15:23.220949+00:00
INFO:__main__:Writes stored successfully for thread_id: 1, ts: 2024-08-05T11:15:23.221984+00:00
INFO:__main__:Data stored successfully under key: checkpoint:1:2024-08-05T11:15:23.234523+00:00
INFO:__main__:Writes stored successfully for thread_id: 1, ts: 2024-08-05T11:15:23.234523+00:00
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Data stored successfully under key: checkpoint:1:2024-08-05T11:15:24.252761+00:0

In [8]:
res

{'messages': [HumanMessage(content="what's the weather in sf", id='eea661b1-4602-4140-8a71-97909de26aee'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jLfonr3BH0UvfUKc5g0huuSL', 'function': {'arguments': '{"city":"sf"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 57, 'total_tokens': 71}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3cd8b62c3b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2f538aa9-ff37-4f0f-b4a0-fc33a6bfc87f-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_jLfonr3BH0UvfUKc5g0huuSL', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, 'output_tokens': 14, 'total_tokens': 71}),
  ToolMessage(content="It's always sunny in sf", name='get_weather', id='055823b2-3be9-435c-a05a-475fff942c2e', tool_call_id='call_jLfonr3BH0UvfUKc5g0huuSL'),
  AIMessage(content='The weather in San Francisco is currently sunny.'

In [9]:
checkpointer.get(config)

INFO:__main__:Data retrieved successfully for key: checkpoint:1:2024-08-05T11:15:24.252761+00:00
INFO:__main__:Checkpoint parsed successfully for thread_id: 1


{'v': 1,
 'ts': '2024-08-05T11:15:24.252761+00:00',
 'id': '1ef531c0-2c67-6f82-8003-27e9d58275f2',
 'channel_values': {'messages': [HumanMessage(content="what's the weather in sf", id='eea661b1-4602-4140-8a71-97909de26aee'),
   AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_jLfonr3BH0UvfUKc5g0huuSL', 'function': {'arguments': '{"city":"sf"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 57, 'total_tokens': 71}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_3cd8b62c3b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2f538aa9-ff37-4f0f-b4a0-fc33a6bfc87f-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_jLfonr3BH0UvfUKc5g0huuSL', 'type': 'tool_call'}], usage_metadata={'input_tokens': 57, 'output_tokens': 14, 'total_tokens': 71}),
   ToolMessage(content="It's always sunny in sf", name='get_weather', id='055823b2-3be9-435c-a05a-475fff942c2e

### With a connection

In [10]:
import redis

# Initialize the Redis synchronous direct connection
sync_redis_direct = redis.Redis(host="localhost", port=6379, db=0)

# Initialize the RedisSaver with the synchronous direct connection
checkpointer = RedisSaver(sync_connection=sync_redis_direct)

graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
config = {"configurable": {"thread_id": "2"}}
res = graph.invoke({"messages": [("human", "what's the weather in sf")]}, config)

checkpoint_tuple = checkpointer.get_tuple(config)

INFO:__main__:No checkpoints found for thread_id: 2
INFO:__main__:Data stored successfully under key: checkpoint:2:2024-08-05T11:15:26.024055+00:00
INFO:__main__:Writes stored successfully for thread_id: 2, ts: 2024-08-05T11:15:26.029054+00:00
INFO:__main__:Data stored successfully under key: checkpoint:2:2024-08-05T11:15:26.029054+00:00
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Data stored successfully under key: checkpoint:2:2024-08-05T11:15:27.036467+00:00
INFO:__main__:Writes stored successfully for thread_id: 2, ts: 2024-08-05T11:15:27.037450+00:00
INFO:__main__:Data stored successfully under key: checkpoint:2:2024-08-05T11:15:27.043986+00:00
INFO:__main__:Writes stored successfully for thread_id: 2, ts: 2024-08-05T11:15:27.043986+00:00
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Data stored successfully under key: checkpoint:2:2024-08-05T11:15:27.837961+00:0

## Use async connection

### With a connection pool

In [11]:
# Initialize a synchronous Redis connection pool
async_pool = initialize_async_pool(url="redis://localhost:6379/0")

checkpointer = RedisSaver(async_connection=async_pool)

INFO:__main__:Asynchronous Redis pool initialized with url=redis://localhost:6379/0


In [12]:
graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
config = {"configurable": {"thread_id": "3"}}
res = await graph.ainvoke(
    {"messages": [("human", "what's the weather in nyc")]}, config
)

INFO:__main__:No checkpoints found for thread_id: 3
INFO:__main__:Data stored successfully under key: checkpoint:3:2024-08-05T11:15:28.021390+00:00
INFO:__main__:Writes stored successfully for thread_id: 3, ts: 2024-08-05T11:15:28.026390+00:00
INFO:__main__:Data stored successfully under key: checkpoint:3:2024-08-05T11:15:28.026390+00:00
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Data stored successfully under key: checkpoint:3:2024-08-05T11:15:29.061047+00:00
INFO:__main__:Writes stored successfully for thread_id: 3, ts: 2024-08-05T11:15:29.061047+00:00
INFO:__main__:Data stored successfully under key: checkpoint:3:2024-08-05T11:15:29.068415+00:00
INFO:__main__:Writes stored successfully for thread_id: 3, ts: 2024-08-05T11:15:29.069415+00:00
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Data stored successfully under key: checkpoint:3:2024-08-05T11:15:29.771410+00:0

In [13]:
checkpoint_tuple = await checkpointer.aget_tuple(config)

INFO:__main__:Data retrieved successfully for key: checkpoint:3:2024-08-05T11:15:29.771410+00:00
INFO:__main__:Checkpoint parsed successfully for thread_id: 3


In [14]:
checkpoint_tuple

CheckpointTuple(config={'configurable': {'thread_id': '3'}}, checkpoint={'v': 1, 'ts': '2024-08-05T11:15:29.771410+00:00', 'id': '1ef531c0-6109-63b7-8003-2279817b32cc', 'channel_values': {'messages': [HumanMessage(content="what's the weather in nyc", id='31036c8a-ce5e-414a-8710-b7d52e86b5f5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_RTPP7yBwAJZymyPXnaOWYfxH', 'function': {'arguments': '{"city":"nyc"}', 'name': 'get_weather'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 58, 'total_tokens': 73}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_4e2b2da518', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1d9dde0b-5de9-4f0c-8a94-11db0d38e25b-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'nyc'}, 'id': 'call_RTPP7yBwAJZymyPXnaOWYfxH', 'type': 'tool_call'}], usage_metadata={'input_tokens': 58, 'output_tokens': 15, 'total_tokens': 73}), ToolMessage(content='It might be cloudy i

### Use connection

In [15]:
from redis.asyncio import Redis as AsyncRedis

async with await AsyncRedis(host="localhost", port=6379, db=0) as conn:
    checkpointer = RedisSaver(async_connection=conn)
    graph = create_react_agent(model, tools=tools, checkpointer=checkpointer)
    config = {"configurable": {"thread_id": "4"}}
    res = await graph.ainvoke(
        {"messages": [("human", "what's the weather in nyc")]}, config
    )
    checkpoint_tuples = [c async for c in checkpointer.alist(config)]

INFO:__main__:No checkpoints found for thread_id: 4
INFO:__main__:Data stored successfully under key: checkpoint:4:2024-08-05T11:15:30.030381+00:00
INFO:__main__:Writes stored successfully for thread_id: 4, ts: 2024-08-05T11:15:30.036471+00:00
INFO:__main__:Data stored successfully under key: checkpoint:4:2024-08-05T11:15:30.035379+00:00
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Data stored successfully under key: checkpoint:4:2024-08-05T11:15:30.854234+00:00
INFO:__main__:Writes stored successfully for thread_id: 4, ts: 2024-08-05T11:15:30.855234+00:00
INFO:__main__:Data stored successfully under key: checkpoint:4:2024-08-05T11:15:30.865223+00:00
INFO:__main__:Writes stored successfully for thread_id: 4, ts: 2024-08-05T11:15:30.865732+00:00
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Data stored successfully under key: checkpoint:4:2024-08-05T11:15:31.568987+00:0